In [1]:
# Main import block
import asyncio
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO
import os
import random

# Sneaky scrapper
import cfscrape
# Upload to google sheets
import gspread
# import df2gspread as d2g
import gspread_pandas as gsp
import numpy as np
import pandas as pd
import httpx
import requests
# Upload to yandex
import yadisk
from bs4 import BeautifulSoup
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

warnings.filterwarnings("ignore")

In [2]:
##### Functions to maintain side manipulations #####
def writer(time_out=10):
    # Function for uploading dataframes into the google docs
    def google_upload():
        try:
            # Параметры подключения
            scope = [
                "https://spreadsheets.google.com/feeds",
                "https://www.googleapis.com/auth/drive",
            ]

            json_keyfile_path = "./misc/macro-parser-lme-c2f2972b48fc.json"
            excel_file_path = "./data/multiple_dataframes.xlsx"
            spreadsheet_key = "19XJClFWuwVdKx0R0Hw9owa6B6YJtgr9rgwT_0AJta4k"

            # API токен для подключения
            credentials = ServiceAccountCredentials.from_json_keyfile_name(
                json_keyfile_path, scope
            )
            gc = gspread.authorize(credentials)

            # Читаем файл
            xls = pd.ExcelFile(excel_file_path)

            # Пробегаемся по каждому листу книги и скидываем их в гугл докс
            for sheet_name in xls.sheet_names: # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
                d2g.upload(
                    df, spreadsheet_key, 
                    sheet_name, 
                    credentials=credentials,
                    row_names=True)
                print(f"{sheet_name} uploaded!")
                time.sleep(time_out)

            print("Uploading whole file to Google Sheets completed") #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            
            # for sheet_name in xls.sheet_names: #&&&&&&&&&&&&&&&&&&&&&&&&&&
            #     df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
            #     worksheet = gc.open_by_key(spreadsheet_key).worksheet(sheet_name)

            #     # Получаем все существующие строки из Google Sheets
            #     existing_rows = worksheet.get_all_values()  # Получаем все значения в виде списка списков
            #     last_five_existing_rows = existing_rows[-5:] if len(existing_rows) > 5 else existing_rows

            # # Преобразуем DF и проверяем существующие строки
            # for index, row in df.iterrows():
            #     row_list = row.tolist()
            #     # Преобразуем Timestamp в строку
            #     row_list = [item.strftime('%Y-%m-%d') if isinstance(item, pd.Timestamp) else item for item in row_list]
                
            #     # Проверяем на существование в последних 5 строках
            #     if row_list not in last_five_existing_rows:
            #         try:
            #             worksheet.append_row(row_list)  # Добавляем строку
            #         except Exception as e:
            #             print(f"Ошибка при добавлении строки {row_list}: {e}")  # Обработка ошибки добавления строки
            #     else:
            #         print(f"Строка {row_list} уже существует в последних 5 строках, пропускаем добавление.")

            # print(f"{sheet_name} uploaded!")

            # print("Uploading whole file to Google Sheets completed")  # $$$$$

        except Exception as error:
            print(error)
            

    # Выгрузка на диск
    def yandex_upload():
        token = pd.read_csv("./misc/token.csv", header=None)
        token = token.iloc[0, 0]

        y = yadisk.YaDisk(token=token)

        try:
            y.upload("./data/multiple_dataframes.xlsx", "/macro_db/database.xlsx")
            

        except Exception as error:
            print(error)
            y.remove("/macro_db/database.xlsx")
            y.upload("./data/multiple_dataframes.xlsx", "/macro_db/database.xlsx")

    # Открываем все необходимые фреймы
    lme = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
    kitko = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
    lbma = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)
    nbk = pd.read_excel("./data/nbk_tenge.xlsx", index_col=0)
    shmet = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
    westmetall = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

    usd = pd.read_excel("./data/cb_currency.xlsx", sheet_name="USD", index_col=0)
    eur = pd.read_excel("./data/cb_currency.xlsx", sheet_name="EUR", index_col=0)
    ausd = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Australian_Dollar", index_col=0
    )
    yuan = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="China_Yuan", index_col=0
    )
    pound = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="British_Pound", index_col=0
    )
    tenge = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Kazakhstan_Tenge", index_col=0
    )
    yen = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Japanese_Yen", index_col=0
    )
    swiss = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Swiss_Franc", index_col=0
    )

    cb_metalls = pd.read_excel(
        "./data/cb_metalls.xlsx", sheet_name="cb_metalls", index_col=0
    )
    
    # Мержим всю собранную инфу в локальную "БД"
    with pd.ExcelWriter(
        "./data/multiple_dataframes.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
        engine="xlsxwriter",
    ) as writer:
        lme.to_excel(writer, sheet_name="LME_non_ferrous", index=False)
        kitko.to_excel(writer, sheet_name="KITCO", index=False)
        lbma.to_excel(writer, sheet_name="LBMA", index=False)
        nbk.to_excel(writer, sheet_name="nbk_tenge", index=False)
        shmet.to_excel(writer, sheet_name="SHMET", index=False)
        westmetall.to_excel(writer, sheet_name="LME_westmetall", index=False)
        usd.to_excel(writer, sheet_name="USD", index=False)
        eur.to_excel(writer, sheet_name="EUR", index=False)
        ausd.to_excel(writer, sheet_name="Australian_Dollar", index=False)
        yuan.to_excel(writer, sheet_name="China_Yuan", index=False)
        pound.to_excel(writer, sheet_name="British_Pound", index=False)
        tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge", index=False)
        yen.to_excel(writer, sheet_name="Japanese_Yen", index=False)
        swiss.to_excel(writer, sheet_name="Swiss_Franc", index=False)
        cb_metalls.to_excel(writer, sheet_name="cb_metalls", index=False)

    print("Database uploaded on yandex.disk")

    yandex_upload()
    google_upload()


# Timeout class for reattempting connection
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")

    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


# Function to convert date from json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def date_format_reverse():
    date_now = date.today()
    date_string = date_now.strftime("%Y-%m-%d %H:%M:%S")
    date_raw = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").timestamp()
    date_raw = date_raw * 1000
    return int(date_raw)


# Session creation via proxy
def get_session(url):
    # Free proxy function
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find("table").find_all("tr")  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all("td"))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == "yes":  # taking only https
                    proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # create session
    session = requests.Session()

    # random proxy
    proxy = get_free_proxies()
    counter = 0

    while counter <= len(proxy):
        try:
            with time_limit(7):
                random_proxy = proxy.sample().values[0]
                session.proxies = {"https": random_proxy}
                response = session.get(url)
                break

        except OSError:
            counter += 1
            pass

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1
            print(f"Attempt {counter+1}")

    return response


async def fetch_html(url, session):
    async with session.get(url) as response:
        return await response.text()
    
    
######

# Free proxy function
def get_free_proxies():
    url = "https://free-proxy-list.net/"
    soup = BeautifulSoup(requests.get(url).content, "html.parser")

    raw_list = []
    proxies = dict()
    trs = soup.find("table").find_all("tr")  # main table

    for i in trs[1:]:
        raw_list.append(i.find_all("td"))  # list of raw data rows

    for i in range(len(raw_list)):  # creating working proxy list
        try:
            if raw_list[i][6].text == "yes":  # taking only https
                proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
        except IndexError:
            continue

    adress = pd.Series(proxies)  # creating proxy series

    return adress

In [3]:
# Check function
def db_check():
    # Базовая функция проверки
    def check_df(df_1, df_2):
        last_row_df_1 = df_1.iloc[-1][0]
        last_row_df_1_data = pd.DataFrame(df_1.iloc[-1]).T.reset_index(drop=True)

        last_row_df_2 = df_2.iloc[-1][0]
        last_row_df_2_data = pd.DataFrame(df_2.iloc[-1]).T.reset_index(drop=True)

        if  last_row_df_1 < last_row_df_2:
            # Добавление недостающей информации из df2 в df1
            df_1_checked = pd.concat([df_1, last_row_df_2_data], ignore_index=True)
            df_2_checked = df_2.copy()

        elif last_row_df_2 < last_row_df_1:
            # Добавление недостающей информации из df2 в df1
            df_2_checked = pd.concat([df_2, last_row_df_1_data], ignore_index=True)
            df_1_checked = df_1.copy()
            
        else:
            # Данные совпадают, ничего не делаем
            df_1_checked = df_1.copy()
            df_2_checked = df_2.copy()

        return df_1_checked, df_2_checked

    # Проверка LME/Westmetall

    lme = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
    westmetall = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

    lme_checked, westmetall_checked = check_df(lme, westmetall)

    if lme.equals(lme_checked) and westmetall.equals(westmetall_checked):

        pass

    else:
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_checked.to_excel(writer, sheet_name="LME_main")

        with pd.ExcelWriter(
            "./data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            westmetall_checked.to_excel(writer, sheet_name="LME_westmetall")

        display(lme_checked, westmetall_checked)

    # Проверка KITCO/LBMA

    kitco = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
    lbma = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

    kitco_checked, lbma_checked = check_df(kitco, lbma)

    if kitco.equals(kitco_checked) and lbma.equals(lbma_checked):

        pass

    else:
        with pd.ExcelWriter(
            "./data/kitko_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            kitco_checked.to_excel(writer, sheet_name="kitco_metall")

        with pd.ExcelWriter(
            "./data/lbme_kitco_subs.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lbma_checked.to_excel(writer, sheet_name="lbme_metall")

        display(kitco_checked, lbma_checked)
        

In [4]:
# Конвертация фреймов в csv
def excel_to_csv_db():

    def df_to_csv(file_name):
        pd.read_excel(f"./data/{file_name}.xlsx", index_col=0).to_csv(
            f"./data/{file_name}.csv", sep=",", index=False
        )
        
    def list_xlsx_files_in_directory(directory):
        try:
            # Получаем список всех файлов и папок
            all_files = os.listdir(directory)

            # Фильтруем только файлы с расширением .xlsx
            xlsx_files = [
                f
                for f in all_files
                if os.path.isfile(os.path.join(directory, f)) and f.endswith(".xlsx")
            ]

            return xlsx_files
        
        except Exception as e:
            print(f"Произошла ошибка: {e}")
            return []


    # Путь к папке
    directory_path = "./data"  

    # Вызов функции
    xlsx_files = list_xlsx_files_in_directory(directory_path)
    xlsx_files = [file.replace(".xlsx","") for file in xlsx_files]

    for file in xlsx_files:
        df_to_csv(file)
    

In [5]:
# Check function
def db_check():
    # Базовая функция проверки
    def check_df(df_1, df_2):
        last_row_df_1 = df_1.iloc[-1]
        last_row_df_2 = df_2.iloc[-1]

        # Сравниваем последние строки
        if last_row_df_1[0] < last_row_df_2[0]:
            # Добавление недостающей информации из df2 в df1
            df_1_checked = pd.concat(
                [df_1, last_row_df_2.to_frame().T], ignore_index=True
            )  # Добавляем строку df2 в df1
            df_2_checked = df_2.copy()
        elif last_row_df_2[0] < last_row_df_1[0]:
            # Добавление недостающей информации из df1 в df2
            df_2_checked = pd.concat(
                [df_2, last_row_df_1.to_frame().T], ignore_index=True
            )  # Добавляем строку df1 в df2
            df_1_checked = df_1.copy()
        else:
            # Данные совпадают, ничего не делаем
            df_1_checked = df_1.copy()
            df_2_checked = df_2.copy()

        return df_1_checked, df_2_checked

    # Проверка LME/Westmetall
    lme = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
    westmetall = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

    lme_checked, westmetall_checked = check_df(lme, westmetall)

    if lme.equals(lme_checked) and westmetall.equals(westmetall_checked):
        pass
    else:
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_checked.to_excel(writer, sheet_name="LME_main")

        with pd.ExcelWriter(
            "./data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            westmetall_checked.to_excel(writer, sheet_name="LME_westmetall")

        display(lme_checked, westmetall_checked)

    # Проверка KITCO/LBMA
    kitco = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
    lbma = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

    kitco_checked, lbma_checked = check_df(kitco, lbma)

    if kitco.equals(kitco_checked) and lbma.equals(lbma_checked):
        pass
    else:
        with pd.ExcelWriter(
            "./data/kitko_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            kitco_checked.to_excel(writer, sheet_name="kitco_metall")

        with pd.ExcelWriter(
            "./data/lbme_kitco_subs.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lbma_checked.to_excel(writer, sheet_name="lbme_metall")

        display(kitco_checked, lbma_checked)

----

### This is the main function block ###

In [6]:

################################################################
##########################   LME ###############################
################################################################
# Асинхронная функция
async def lme_selenium_async(random_proxy=True):
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"
    
    # service = Service()
    # options = webdriver.ChromeOptions()
    # options.add_argument('headless') # без графического интерфейса
    # options.add_argument('--no-sandbox')  # для будущей контейнеризации
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    # driver = webdriver.Chrome(service=service, options=options)
    
    if random_proxy:
        # Получаем список прокси
        proxies = get_free_proxies()
        print(f"Найдено {len(proxies)} HTTPS-прокси")
        
        driver = None
        timeout_seconds = 7
        max_attempts = len(proxies) + 1  # +1 - чтобы попробовать без прокси в конце
        attempt = 0

        while attempt < max_attempts:
            options = webdriver.ChromeOptions()
            options.add_argument('headless')
            options.add_argument('--no-sandbox')
            options.add_argument(
                "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
            )

            proxy = None
            if attempt < len(proxies):
                proxy = proxies[attempt]
                print(f"Попытка с прокси: {proxy}")
                options.add_argument(f'--proxy-server={proxy}')
            else:
                print("Попытка без использования прокси")

            try:
                service = Service()
                driver = webdriver.Chrome(service=service, options=options)

                # Устанавливаем общий тайм-аут ожидания
                driver.set_page_load_timeout(timeout_seconds)
                
                # Попытка загрузки страницы
                driver.get(url)

                # Если всё прошло успешно - выходим из цикла
                print("Успешное подключение!")
                break

            except (TimeoutException, WebDriverException) as e:
                print(f"Ошибка при подключении (попытка {attempt + 1}): {e}")
                if driver:
                    driver.quit()
                attempt += 1
                continue

        if not driver:
            print("Не удалось подключиться ни через один прокси.")
            return

    else:
        service = Service()
        
        # Headless-block
        # options = webdriver.ChromeOptions()
        # options.add_argument('headless') # без графического интерфейса
        # options.add_argument('--no-sandbox')  # для будущей контейнеризации
        # options.add_argument(
        #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        # driver = webdriver.Chrome(service=service, options=options)

        # Robust impl
        driver = webdriver.Chrome(service=service)
    
    # driver = webdriver.Chrome(service=service)

    try:
        # driver.maximize_window()
        driver.get(url=url)

        time.sleep(5)

        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )
        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )

        time.sleep(5)

        html_code = driver.page_source
        # driver.close()

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        # Приводим к тексту
        metalls_raw = data_raw[0].text

        # Убираем все лишнее
        metalls_raw = metalls_raw.replace(
            " ", "").replace("LME", "LME_").split("\n")
        metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # Конфигурируем новый лист по металлам
        metalls_raw = [metalls_raw[i: i + 4]
                       for i in range(0, len(metalls_raw), 4)]

        # Обрезаем до нужной инфы
        metalls_raw = [i[:2] for i in metalls_raw]

        # Приведем цены к числовому формату
        for metall in metalls_raw:
            metall[1] = pd.to_numeric(metall[1])

        # Формируем строку датафрейма по металлам
        dict_list = [{item[0]: item[1]} for item in metalls_raw]

        metall_df = pd.DataFrame()

        for i in dict_list:
            keys = list(i.keys())
            values = list(i.values())
            metall_df[keys[0]] = values

        # Дропаем ненужные колонки
        metall_df.drop(columns=["LME_AluminiumAlloy",
                       "LME_NASAAC"], inplace=True)
        metall_df = metall_df.rename(
            columns={
                "LME_Aluminium": "aluminium",
                "LME_Copper": "copper",
                "LME_Lead": "lead",
                "LME_Nickel": "nickel",
                "LME_Zinc": "zink",
                "LME_Tin": "tin",
            }
        )

        # Достанем в вставим дату
        date_raw = soup.find_all(
            "span", class_="metal-block-container__refreshed-on")
        date_raw = date_raw[0]
        date_raw = str(date_raw).split(">")[1]
        date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        metall_df["date"] = date_raw

        # Поменяем порядок колонок
        new_column_order = [
            "date",
            "aluminium",
            "copper",
            "lead",
            "nickel",
            "zink",
            "tin",
        ]
        metall_df = metall_df[new_column_order]

        # Сохраняем полученные результаты
        lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
        lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

        # Дропаем дубликаты
        lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

        # Сохраняем
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_db.to_excel(writer, sheet_name="LME_main")

        print("LME_main is done!!!")
        # display(lme_db.tail(10))

    except Exception as error:
        print(error)

    finally:
        driver.close()
        driver.quit()

In [7]:
###############################################################
########################   antimony ###########################
###############################################################

async def antimony():
    url = "https://www.metal.com/Antimony/201102250328"

    async def get_raw_data(url):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            # Парсинг HTML с помощью BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Извлечение цены
            prices = [
                float(i.text.replace(",", ""))
                for i in soup.find_all("div", class_="price___2mpJr")
            ][1:]
            
            cny_price = prices[1]
            usd_price = prices[0]

            # Извлечение даты
            response_date = soup.find("div", class_="date___2QLQE")
            date = [pd.to_datetime(response_date.text)]

            # Создание DataFrame
            result = pd.DataFrame(
                {
                    "Date": date,
                    "Avg(CNY/mt,VAT included)": cny_price,
                    "Avg With Rate(USD/mt,VAT included)": usd_price,
                }
            )
            
            historical_df = pd.read_excel("./data/antimony.xlsx", index_col=0)
            
            historical_df = (
                pd.concat([historical_df, result], axis=0)
                .reset_index(drop=True)
                .drop_duplicates()
            )
            
            with pd.ExcelWriter(
                        "./data/antimony.xlsx",
                        date_format="YYYY-MM-DD",
                        datetime_format="YYYY-MM-DD",
                    ) as writer:
                        historical_df.to_excel(writer, sheet_name="antimony")

            # print("antimony is done!!!")

            return historical_df

    try:
        # Получаем данные асинхронно
        antimony = await get_raw_data(url)
        print("antimony is done!!!")

    except Exception as error:
        print("Произошла ошибка:", str(error))

# # Запускаем асинхронную функцию
# if __name__ == "__main__":
#     await antimony() # Используйте asyncio.run для запуска в обычном скрипте

In [8]:
# ###############################################################
# ########################   KITCO (main)-old ########################
# ###############################################################


# # Ассинхронный парсер Kitco
# async def kitco_parser_async():
#     async def page_getter():
#         try:
#             url = "https://www.kitco.com/price/fixes/london-fix"

#             service = Service()
#             driver = webdriver.Chrome(service=service)
#             # driver.maximize_window()
#             driver.get(url=url)
#             time.sleep(2)

#             driver.execute_script(
#                 "window.scrollTo(0, window.scrollY + window.innerHeight);"
#             )
#             driver.execute_script(
#                 "window.scrollTo(0, window.scrollY + window.innerHeight);"
#             )

#             time.sleep(2)

#             html_code = driver.page_source
#             #driver.close()

#             return html_code

#         except Exception as error:
#             print(error)

#         finally:
#             driver.close()
#             driver.quit()

#     html_code = await page_getter()

#     try:
#         # Работа над полученными данными
#         soup = BeautifulSoup(html_code)

#         day = soup.find_all("div", class_="grid")

#         # Вычленение первичного контента
#         day_content = []
#         for i in range(len(day)):
#             day_content.append(day[i].contents)

#         # Первичное преобразование
#         for container in day_content:
#             for element in range(len(container)):
#                 try:
#                     container[element] = (
#                         str(container[element])
#                         .replace("<div>", "")
#                         .replace("</div>", "")
#                         .replace("<!-- -->", "")
#                     )
#                     container[element]
#                 except:
#                     pass

#         # Удалим неинформативные строки
#         day_content = day_content[1:]
#         day_content.pop(1)

#         # Преобразование дат и вычлинение значений
#         for element in day_content:
#             # Преобразованиие даты
#             element[0] = pd.to_datetime(element[0]).strftime("%Y-%m-%d")

#             # Числовые преобразования
#             element[1] = pd.to_numeric(
#                 element[1].split("/")[1].strip().replace(",", "")
#             )
#             element[2] = pd.to_numeric(
#                 element[2].split("/")[0].strip().replace(",", "")
#             )
#             element[3] = pd.to_numeric(
#                 element[3].split("/")[1].strip().replace(",", "")
#             )
#             element[4] = pd.to_numeric(
#                 element[4].split("/")[1].strip().replace(",", "")
#             )

#         # Формируем датафрейм
#         day_content = pd.DataFrame(
#             day_content, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
#         )

#         # Преобразуем типы
#         day_content["Date"] = pd.to_datetime(day_content["Date"])
#         day_content = day_content.sort_values("Date").reset_index(drop=True)

#         # Возьмем последние 5 строк
#         day_update = day_content.tail()

#         # Склеим с историческими данными
#         historical_df = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
#         historical_df = (
#             pd.concat([historical_df, day_update], axis=0)
#             .reset_index(drop=True)
#             .drop_duplicates()
#         )

#         with pd.ExcelWriter(
#             "./data/kitko_db.xlsx",
#             date_format="YYYY-MM-DD",
#             datetime_format="YYYY-MM-DD",
#         ) as writer:
#             historical_df.to_excel(writer, sheet_name="kitco_metall")

#         print("KITKO_main is done!!!")
#         # display(historical_df.tail(10))

#     except Exception as e:
#         print(e)

In [9]:
################################################################
#########################   KITCO (new) ########################
################################################################

async def kitco_parser_async():
    url = "https://www.kitco.com/price/fixes/london-fix"

    async def get_raw_data(url):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            # Парсинг HTML с помощью BeautifulSoup
            soup = BeautifulSoup(
                response.text, "html.parser"
            )  # Указание парсера, например, 'html.parser'
            
            day = soup.find_all("div", class_="grid")
            
            # Вычленение первичного контента
            day_content = []
            
            for i in range(len(day)):
                day_content.append(day[i].contents)

            # Первичное преобразование
            for container in day_content:
                for element in range(len(container)):
                    try:
                        container[element] = (
                            str(container[element])
                            .replace("<div>", "")
                            .replace("</div>", "")
                            .replace("<!-- -->", "")
                        )
                        container[element]
                    except Exception as e:
                        print(e)

            # Удалим неинформативные строки
            day_content = day_content[1:]
            day_content.pop(1)

            # Преобразование дат и вычлинение значений
            for element in day_content:
                # Преобразованиие даты
                element[0] = pd.to_datetime(element[0]).strftime("%Y-%m-%d")

                # Числовые преобразования
                element[1] = pd.to_numeric(
                    element[1].split("/")[1].strip().replace(",", "")
                )
                element[2] = pd.to_numeric(
                    element[2].split("/")[0].strip().replace(",", "")
                )
                element[3] = pd.to_numeric(
                    element[3].split("/")[1].strip().replace(",", "")
                )
                element[4] = pd.to_numeric(
                    element[4].split("/")[1].strip().replace(",", "")
                )

            # Формируем датафрейм
            day_content = pd.DataFrame(
                day_content, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
            )

            # Преобразуем типы
            day_content["Date"] = pd.to_datetime(day_content["Date"])
            day_content = day_content.sort_values("Date").reset_index(drop=True)

            # Возьмем последние 5 строк
            day_update = day_content.tail()
            
            # Склеим с историческими данными
            historical_df = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
            historical_df = (
                pd.concat([historical_df, day_update], axis=0)
                .reset_index(drop=True)
                .drop_duplicates(subset=["Date"], keep="last")
            )
            historical_df.sort_values(by="Date", inplace=True)
            historical_df.reset_index(drop=True, inplace=True)

            with pd.ExcelWriter(
                        "./data/kitko_db.xlsx",
                        date_format="YYYY-MM-DD",
                        datetime_format="YYYY-MM-DD",
                    ) as writer:
                        historical_df.to_excel(writer, sheet_name="kitco_metall")

            print("KITKO_main is done!!!")
            # display(historical_df.tail(10))

            return historical_df

    try:
        # Получаем данные асинхронно
        gold = await get_raw_data(url)
        # display(gold)

    except Exception as error:
        print("Произошла ошибка:", error)


# Запускаем асинхронную функцию
# await lbma_precious_async()  # Просто вызываем, без asyncio.run()


In [10]:
################################################################
#########################   LBMA (KITCO subs) ##################
################################################################


# async def lbma_prescious_async():
#     url_gold = "https://prices.lbma.org.uk/json/gold_pm.json"
#     url_silver = "https://prices.lbma.org.uk/json/silver.json"
#     url_platinum = "https://prices.lbma.org.uk/json/platinum_pm.json"
#     url_paladium = "https://prices.lbma.org.uk/json/palladium_pm.json"

#     async def get_raw_data(url, metall="Default"):
#         scraper = cfscrape.create_scraper()
#         scraped_data = scraper.get(url)

#         # scraped_data = get_session(url=url)

#         raw_data = pd.read_json(scraped_data.text)
#         data = raw_data[["d", "v"]]
#         data["v"] = data["v"].apply(lambda x: x[0])
#         data["d"] = pd.to_datetime(data["d"])

#         data.rename(columns={"d": "Date", "v": metall}, inplace=True)
#         data = data.tail(4)

#         return data

#     try:
#         gold = await get_raw_data(url_gold, metall="Gold")
#         silver = await get_raw_data(url_silver, metall="Silver")
#         platinum = await get_raw_data(url_platinum, metall="Platinum")
#         paladium = await get_raw_data(url_paladium, metall="Palladium")

#         result_df = (
#             gold.merge(silver, on="Date", how="outer")
#             .merge(platinum, on="Date", how="outer")
#             .merge(paladium, on="Date", how="outer")
#         )

#         result_df.fillna(value=0, inplace=True)
#         result_df = result_df.sort_values(by="Date")
#         result_df.reset_index(inplace=True, drop=True)

#         historical = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

#         result = pd.concat([historical, result_df], axis=0).reset_index(drop=True)

#         result.drop_duplicates(inplace=True)

#         result = result.sort_values(by="Date")

#         result.to_excel("./data/lbme_kitco_subs.xlsx", sheet_name="lbme_metall")

#         print("LBMA is done!!!")
#         # display(result.tail(10))

#     except Exception as error:
#         print(error)
        
        
    

In [11]:
################################################################
#########################   LBMA (KITCO subs)-new ##################
################################################################


async def lbma_prescious_async():
    url_gold = "https://prices.lbma.org.uk/json/gold_pm.json"
    url_silver = "https://prices.lbma.org.uk/json/silver.json"
    url_platinum = "https://prices.lbma.org.uk/json/platinum_pm.json"
    url_paladium = "https://prices.lbma.org.uk/json/palladium_pm.json"

    async def get_raw_data(url, metall="Default"):
        async with httpx.AsyncClient() as client:
            response = await client.get(url)  # Асинхронный запрос
            response.raise_for_status()  # Проверка на ошибки

            raw_data = pd.read_json(response.text)
            data = raw_data[["d", "v"]]
            data["v"] = data["v"].apply(lambda x: x[0])
            data["d"] = pd.to_datetime(data["d"])

            data.rename(columns={"d": "Date", "v": metall}, inplace=True)
            return data.tail(10)

    try:
        # Получаем данные асинхронно
        gold = await get_raw_data(url_gold, metall="Gold")
        silver = await get_raw_data(url_silver, metall="Silver")
        platinum = await get_raw_data(url_platinum, metall="Platinum")
        paladium = await get_raw_data(url_paladium, metall="Palladium")

        # Теперь это DataFrame
        result_df = (
            gold.merge(silver, on="Date", how="outer")
            .merge(platinum, on="Date", how="outer")
            .merge(paladium, on="Date", how="outer")
        )

        result_df.fillna(value=0, inplace=True)
        result_df = result_df.sort_values(by="Date")
        result_df.reset_index(inplace=True, drop=True)

        historical = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

        result = pd.concat([historical, result_df], axis=0).reset_index(drop=True)

        result.drop_duplicates(inplace=True)

        result = result.sort_values(by="Date")

        result.to_excel("./data/lbme_kitco_subs.xlsx", sheet_name="lbme_metall")
        # display(result)
        print("LBMA is done!!!")

    except Exception as error:
        print("Произошла ошибка:", error)

# Пример запуска асинхронной функции
# await lbma_prescious_async()


In [12]:
################################################################
####################   CB_currencies ###########################
################################################################


async def cb_currency():
    async def get_currency(currency):
        current_date = datetime.today().strftime("%d.%m.%Y")

        dict_of_currencies = {
            "USD": "R01235",
            "EUR": "R01239",
            "Australian_Dollar": "R01010",
            "China_Yuan": "R01375",
            "British_Pound": "R01035",
            "Kazakhstan_Tenge": "R01335",
            "Japanese_Yen": "R01820",
            "Swiss_Franc": "R01775",
        }

        url = f"https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.\
Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.\
date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={dict_of_currencies[currency]}&UniDbQuery.\
From=01.01.2023&UniDbQuery.To={current_date}"

        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        return scraped_data

    def data_reconstruction(scraped_data, name):

        # Получаем датафрейм
        data = pd.read_html(scraped_data.text.replace(",", "."))[0]
        data.columns = data.iloc[1]

        # Преобразуем
        data = data.iloc[2:]
        data = data.rename(
            columns={
                data.columns[0]: "date",
                data.columns[1]: "unit",
                data.columns[2]: "nominal",
            }
        ).reset_index(drop=True)

        # Cкидываем название индекса
        data = pd.read_csv(StringIO(data.to_string(index=False)), delim_whitespace=True)
        data["date"] = pd.to_datetime(data["date"], format="%d.%m.%Y")
        data["unit"] = pd.to_numeric(data["unit"])
        data["nominal"] = pd.to_numeric(data["nominal"])

        data = data.sort_values("date").reset_index(drop=True)

        print(f"{name} is done!")
        # display(data.tail())

        return data

    try:
        raw_usd = await get_currency(currency="USD")
        raw_eur = await get_currency(currency="EUR")
        raw_ausd = await get_currency(currency="Australian_Dollar")
        raw_yuan = await get_currency(currency="China_Yuan")
        raw_pound = await get_currency(currency="British_Pound")
        raw_tenge = await get_currency(currency="Kazakhstan_Tenge")
        raw_yen = await get_currency(currency="Japanese_Yen")
        raw_swiss = await get_currency(currency="Swiss_Franc")

        usd = data_reconstruction(raw_usd, name="USD")
        eur = data_reconstruction(raw_eur, name="EUR")
        ausd = data_reconstruction(raw_ausd, name="Australian_Dollar")
        yuan = data_reconstruction(raw_yuan, name="China_Yuan")
        pound = data_reconstruction(raw_pound, name="British_Pound")
        tenge = data_reconstruction(raw_tenge, name="Kazakhstan_Tenge")
        yen = data_reconstruction(raw_yen, name="Japanese_Yen")
        swiss = data_reconstruction(raw_swiss, name="Swiss_Franc")

        with pd.ExcelWriter(
            "./data/cb_currency.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            usd.to_excel(writer, sheet_name="USD")
            eur.to_excel(writer, sheet_name="EUR")
            ausd.to_excel(writer, sheet_name="Australian_Dollar")
            yuan.to_excel(writer, sheet_name="China_Yuan")
            pound.to_excel(writer, sheet_name="British_Pound")
            tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge")
            yen.to_excel(writer, sheet_name="Japanese_Yen")
            swiss.to_excel(writer, sheet_name="Swiss_Franc")

    except Exception as error:
        print(error)
        

In [13]:
################################################################
####################   CB_metalls ##############################
################################################################


async def cb_metalls():
    try:
        # Первичные гиперпараметры
        current_year = datetime.now().year

        url = f"https://www.cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted\
=True&UniDbQuery.From=01.01.2023&UniDbQuery.To=31.12.{current_year}&UniDbQuery.Gold\
=true&UniDbQuery.Silver=true&UniDbQuery.Platinum=true&UniDbQuery.Palladium\
=true&UniDbQuery.so=1"

        # Запуск скраппера
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # Первичное получение данных
        metalls = pd.read_html(scraped_data.text.replace(",", "."))[0]

        # Переименовывание колонок
        metalls = metalls.rename(
            columns={
                metalls.columns[0]: "date",
                metalls.columns[1]: "gold",
                metalls.columns[2]: "silver",
                metalls.columns[3]: "platinum",
                metalls.columns[4]: "palladium",
            }
        )

        # Приведение данных к необходимому формату
        metalls["date"] = pd.to_datetime(metalls["date"], format="%d.%m.%Y")
        metalls["gold"] = pd.to_numeric(metalls["gold"].str.replace(" ", ""))
        metalls["silver"] = pd.to_numeric(metalls["silver"])
        metalls["platinum"] = pd.to_numeric(metalls["platinum"].str.replace(" ", ""))
        metalls["palladium"] = pd.to_numeric(metalls["palladium"].str.replace(" ", ""))

        metalls = metalls.sort_values("date").reset_index(drop=True)

        with pd.ExcelWriter(
            "./data/cb_metalls.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            metalls.to_excel(writer, sheet_name="cb_metalls")

        print("CB_metalls parsing is DONE!")
        # display(metalls.tail(20))

    except Exception as error:
        print(error)


In [14]:
################################################################
#########################   NBK ################################
################################################################

async def nbk_tenge_async():
    # Realy unrelieable source, mb it would be better off with using ms query inside the file
    try:
        year = date.today().year

        upper_bound = f'01.01.2022'
        lower_bound = f'31.12.{year}'

        url = f'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut\
            /report?rates%5B%5D=5&beginDate={upper_bound}&endDate={lower_bound}'

        counter = 0

        while counter <= 6:
            try:
                with time_limit(15):
                    page = requests.get(url=url)
                    break

            except TimeoutException:
                # print("NBK_tenge timed out! Another attempt")
                counter += 1

        temp_df = pd.read_html(page.text)
        df = temp_df[0]
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])  # , dayfirst=True)
        df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

        with pd.ExcelWriter(
                './data/nbk_tenge.xlsx') as writer:
            df.to_excel(writer, sheet_name='tenge')

        print('NBK_tenge parsing is DONE!')
        # display(df.tail(10))

        return df
    
    except Exception as error:
        print(error)

In [15]:
################################################################
##########################  SHMET ##############################
################################################################

async def shmet_optimized_async():
    url = "https://en.shmet.com/api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0"
    
    try:
        responce = requests.get(url)

        day_df = pd.DataFrame(responce.json()["data"])
        day_df["date"] = date.today()
        cooper_row = day_df[day_df["name"].str.contains("cu", case=False)]

        result = cooper_row[["date", "middle", "unit"]]
        result["date"] = pd.to_datetime(result["date"])
        result = result.rename(columns={"middle": "price"})

        hist_data = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)

        new_df = pd.concat([result, hist_data], axis=0).reset_index(drop=True)
        new_df.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
            "./data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            new_df.to_excel(writer, sheet_name="SHMET")

        final = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
        final.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
            "./data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            final.to_excel(writer, sheet_name="SHMET")

        print("SHMET is done!!!")
        # display(final.head(10))
    
    except Exception as error:
        print(error)


In [16]:
################################################################
################## NEW_WESTMETALL###############################
################################################################


async def new_westmetall_async():
    async def get_data(metall, col_name):
        url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash"
        response = requests.get(url=url)

        df = pd.read_html(response.text)[0][:30]
        data = df.iloc[:, :2]

        data = data.query("date != 'date'")
        data["date"] = pd.to_datetime(data["date"])
        data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
        data.rename(columns={data.columns[1]: col_name}, inplace=True)

        return data

    try:
        al = await get_data(metall="Al", col_name="aluminium")
        cu = await get_data(metall="Cu", col_name="copper")
        pb = await get_data(metall="Pb", col_name="lead")
        ni = await get_data(metall="Ni", col_name="nickel")
        zn = await get_data(metall="Zn", col_name="zink")
        tn = await get_data(metall="Sn", col_name="tin")

        result = (
            pd.merge(al, cu, on="date", how="left")
            .merge(pb, on="date", how="left")
            .merge(ni, on="date", how="left")
            .merge(zn, on="date", how="left")
            .merge(tn, on="date", how="left")
        )

        old_data = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

        final_data = pd.concat([old_data, result], axis=0)

        final_data.drop_duplicates(subset="date", inplace=True)

        final_data.sort_values(by="date", inplace=True)

        final_data.reset_index(inplace=True, drop=True)

        with pd.ExcelWriter(
            "./data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            final_data.to_excel(writer, sheet_name="LME_westmetall")

        print("WESTMETALL is done!!!")
        # display(final_data.tail(10))

    except Exception as error:
        print(error)


In [17]:
if __name__ == "__main__":
    
    print("Parsing started...")
    
    await asyncio.gather(
        kitco_parser_async(),
        lbma_prescious_async(),
        antimony(),
        lme_selenium_async(random_proxy=False),
        new_westmetall_async(),
        cb_currency(),
        cb_metalls(),
        nbk_tenge_async(),
        shmet_optimized_async(),
    )
    print("All tasks are done!")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Checking DB...")
    db_check()
    print("DB check completed!")
    
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Visual control")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

    kitco = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
    print("kitko_db")
    display(kitco.tail(5))

    lbma = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)
    print("lbma_prescious_db")
    display(lbma.tail(5))

    lme_selenium = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
    print("lme_selenium_db")
    display(lme_selenium.tail(5))

    new_westmetall = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)
    print("new_westmetall_db")
    display(new_westmetall.tail(5))
    
    antimony_df = pd.read_excel("./data/antimony.xlsx", index_col=0)
    print("antimony_db")
    display(antimony_df.tail(5))

    cb_currency_df = pd.read_excel("./data/cb_currency.xlsx", index_col=0)
    print("cb_currency_db")
    display(cb_currency_df.tail(5))

    cb_metalls_df = pd.read_excel("./data/cb_metalls.xlsx", index_col=0)
    print("cb_metalls_db")
    display(cb_metalls_df.tail(5))

    nbk_tenge_df = pd.read_excel("./data/nbk_tenge.xlsx", index_col=0)
    print("nbk_tenge_db")
    display(nbk_tenge_df.tail(5))

    shmet_historical_df = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
    print("shmet_historical_db")
    display(shmet_historical_df.head(5))
    
    
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Convertion...")
    excel_to_csv_db()
    print("Complited!")

    # time.sleep(10)
    # Аплоад
    # writer(time_out=7)

Parsing started...
Unable to parse string "OfficialPrice" at position 0
WESTMETALL is done!!!
USD is done!
EUR is done!
Australian_Dollar is done!
China_Yuan is done!
British_Pound is done!
Kazakhstan_Tenge is done!
Japanese_Yen is done!
Swiss_Franc is done!
CB_metalls parsing is DONE!
NBK_tenge parsing is DONE!
HTTPSConnectionPool(host='en.shmet.com', port=443): Max retries exceeded with url: /api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))
Произошла ошибка: Redirect response '302 Found' for url 'https://www.kitco.com/price/fixes/london-fix'
Redirect location: 'http://www.kitco.com/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/302
antimony is done!!!
LBMA is done!!!
All tasks are done!
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++

,date,aluminium,copper,lead,nickel,zink,tin
0,2022-01-04 00:00:00,2815.5,9660.0,2327.0,20730,3602.0,39400
1,2022-01-05 00:00:00,2866.0,9778.0,2343.0,20900,3660.0,39700
2,2022-01-06 00:00:00,2912.5,9565.0,2291.0,20480,3590.0,39650
3,2022-01-07 00:00:00,2919.5,9615.0,2342.0,20725,3602.5,40650
4,2022-01-10 00:00:00,2923.0,9665.0,2303.0,21045,3576.5,40425
...,...,...,...,...,...,...,...
973,2025-11-10 00:00:00,2867.5,10800.0,2043.5,14920,3296.0,36240
974,2025-11-11 00:00:00,2841.0,10777.0,2037.0,14850,3172.0,36300
975,2025-11-12 00:00:00,2856.0,10833.0,2044.0,14845,3179.5,37000
976,2025-11-13 00:00:00,2876.0,10942.0,2058.5,14855,3180.0,37650


,date,aluminium,copper,lead,nickel,zink,tin
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0,39400
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0,39700
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0,39650
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5,40650
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5,40425
...,...,...,...,...,...,...,...
973,2025-11-10,2867.5,10800.0,2043.5,14920,3296.0,36240
974,2025-11-11,2841.0,10777.0,2037.0,14850,3172.0,36300
975,2025-11-12,2856.0,10833.0,2044.0,14845,3179.5,37000
976,2025-11-13,2876.0,10942.0,2058.5,14855,3180.0,37650


,Date,Gold,Silver,Platinum,Palladium
0,2023-01-03 00:00:00,1843.25,24.295,1082.0,1795
1,2023-01-04 00:00:00,1857.3,24.29,1080.0,1736
2,2023-01-05 00:00:00,1834.0,23.41,1062.0,1783
3,2023-01-06 00:00:00,1852.2,23.455,1073.0,1784
4,2023-01-09 00:00:00,1878.85,23.85,1092.0,1793
...,...,...,...,...,...
713,2025-11-10 00:00:00,4090.25,50.04,1576.0,1414
714,2025-11-11 00:00:00,4123.3,51.24,1592.0,1446
715,2025-11-12 00:00:00,4136.75,51.535,1580.0,1439
716,2025-11-13 00:00:00,4195.65,53.87,1602.0,1463


,Date,Gold,Silver,Platinum,Palladium
0,1968-01-02,0.00,2.173,0.0,0.0
1,1968-01-03,0.00,2.225,0.0,0.0
2,1968-01-04,0.00,2.171,0.0,0.0
3,1968-01-05,0.00,2.157,0.0,0.0
4,1968-01-08,0.00,2.163,0.0,0.0
...,...,...,...,...,...
14664,2025-11-10,4090.25,50.040,1576.0,1414.0
14665,2025-11-11,4123.30,51.240,1592.0,1446.0
14666,2025-11-12,4136.75,51.535,1580.0,1439.0
14667,2025-11-13,4195.65,53.870,1602.0,1463.0


...

nbk_tenge_db


,date,Числовое значение,ДОЛЛАР США
1410,2025-11-11,1,523.58
1411,2025-11-12,1,525.28
1412,2025-11-13,1,524.65
1413,2025-11-14,1,524.32
1414,2025-11-15,1,522.19


shmet_historical_db


,date,price,unit
0,2025-11-14,87210,Yuan/MT
2,2025-11-13,86770,Yuan/MT
3,2025-11-11,86530,Yuan/MT
4,2025-11-08,85925,Yuan/MT
5,2025-11-07,86030,Yuan/MT


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Convertion...
Complited!


In [18]:
# writer(time_out=6)